# Final D: Object Pushing

In your final, you will implement a controller for a block pushing task.  Compared to grasping, pushing is more useful for manipulating large objects.  It can also be used to move objects out of reach of normal grasping.  However, the object behavior must be modeled dynamically rather than kinematically, because the object cannot be modeled as rigidly attached due the gripper.  It can also be challenging to predict the object's movement, due to uncertainty in the frictional response with the surface it is resting on.  Often, a combination of planning and feedback control is needed to make pushing work well.

In [18]:
%load_ext autoreload
%autoreload 2
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.io import loader
import math
import random
from IPython.display import clear_output,Markdown
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
fn = "data/ur5Block.xml"
res = world.readFile(fn)
robot = world.robot(0)

# perturb the block configuration
objectTransforms = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]
random.shuffle(objectTransforms)
for i in range(world.numRigidObjects()):
    R,t = objectTransforms[i]
    t[0] += random.uniform(-0.05,0.05)
    t[1] += random.uniform(-0.05,0.05)
    R = so3.from_axis_angle(((0,0,1),random.uniform(0,math.pi*2)))
    world.rigidObject(i).setTransform(R,t)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = 0.64181
qhome[5] = math.pi/2.0

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#set up the controller and useful subroutines for converting to/from Klampt
def controller_to_klampt(qcontroller):
    """Converts a 6+1 controller configuration to Klampt format"""
    qorig = robot.getConfig()
    q = [v for v in qorig]
    for i in range(6):
        q[i+1] = qcontroller[i]
    robot.setConfig(q)
    robot.driver(6).setValue(qcontroller[6])
    q = robot.getConfig()
    robot.setConfig(qorig)
    return q

def klampt_to_controller(qklampt):
    """Converts a Klamp't configuration to 6+1 controller format"""
    qorig = robot.getConfig()
    res = [qklampt[i+1] for i in range(6)]
    robot.setConfig(qklampt)
    res.append(robot.driver(6).getValue())
    robot.setConfig(qorig)
    return res

CONTROLLER = 'simulation'
#when you are ready, replace the above line with this
#CONTROLLER = 'physical'
if CONTROLLER == 'simulation':
    from RobotControllerEmulator import kill_controller_threads,UR5WithGripperController
    controllerWorld = world.copy()
    kvis_ground_truth = ipyklampt.KlamptWidget(controllerWorld,width=600,height=400)
    robot_control_api = UR5WithGripperController(controllerWorld,randomize=True)
    VIS_UPDATE_FREQUENCY = 10
    VIS_UPDATE_ITERS = 0
    def _update_vis_hook():
        global VIS_UPDATE_ITERS
        VIS_UPDATE_ITERS += 1
        if VIS_UPDATE_ITERS % VIS_UPDATE_FREQUENCY == 0:
            #this is called inside the thread
            kvis_ground_truth.update()
    def start_controller():
        kill_controller_threads()
        robot_control_api.start(_update_vis_hook)
    kvis_ground_truth.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})
else:
    from robot_api.RobotController import UR5WithGripperController
    robot_control_api = UR5WithGripperController(host='192.168.0.1')
    def start_controller():
        robot_control_api.start()
def update_vis_from_controller():
    q = robot_control_api.getConfig()
    robot.setConfig(controller_to_klampt(q))
    kvis.update()
def stop_controller():
    robot_control_api.stop()
def wait_controller(duration):
    t0 = time.time()
    tstart = t0
    while time.time()-tstart < duration:
        update_vis_from_controller()
        t1 = time.time()
        time.sleep(max(0,0.05-(t1-t0)))
        t0 = t1


In [20]:
if CONTROLLER == 'simulation':
    display(Markdown("## Ground truth world"))
    display(kvis_ground_truth)

controls = widgets.HBox([])
display(controls)

display(Markdown("## Planning world"))
display(kvis)

output_area = widgets.Output()
display(output_area)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

## Ground truth world

KlamptWidget(camera={u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.309394957176759…

HBox()

## Planning world

KlamptWidget(height=400, scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uu…

Output()

In [17]:
#Call this cell after you started the thread if you want to save CPU
stop_controller()

Stopping UR5WithGripperController emulation thread...
UR5GripperAPI thread terminating
Stopped.


In [13]:
#Call this cell to start the controller thread (the dot in the upper right of the notebook should flicker)
start_controller()
update_vis_from_controller()

UR5WithGripperController emulation thread not started
Starting UR5WithGripperController emulation thread...
UR5GripperAPI thread #1 starting


In [14]:
#assumes perfect sensing
def update_objects_from_simulator():
    global world,controller_world
    for i in xrange(controllerWorld.numRigidObjects()):
        world.rigidObject(i).setTransform(*controllerWorld.rigidObject(i).getTransform())

#target parameters -- you may change these values for testing
target_position = (0.26,-0.25,0.90)
kvis.add_sphere("target",x=target_position[0],y=target_position[1],z=target_position[2],r=0.02)

## Controller implementation

You must implement your controller in the following bare-bones control loop.  You are free to add anything you like, including state machine behavior, inverse kinematics, motion planning, etc.  However, each loop should not take an excessive amount of time.

The controller should move its end effector to the block and then start to push it toward the target position (target_position defined above).  It should return None to stop running.

In [15]:
#TODO: fill me in

from klampt.model import ik

#add whatever global variables you'd like here
t = 0

def push_control_init(robot,q):
    """Initialize any global variables if youd like
    
    To debug during stepping, you can set the robot's configuration and then call kvis.update().  The robot in the
    planning world will be drawn at the given configuration.
    
    Don't "cheat" by using the controllerWorld data structure!  You may use update_objects_from_simulator
    to get the object's position.
    """
    #don't forget to mark global variables as global
    global t
    t = 0

def push_control_loop(robot,q,dt):
    """Returns the next desired configuration of the robot.  Return None to stop the push loop.
    
    To debug during stepping, you can set the robot's configuration and then call kvis.update().  The robot in the
    planning world will be drawn at the given configuration.
    
    Don't "cheat" by using the controllerWorld data structure!  You may use update_objects_from_simulator
    to get the object's updated position.
    """
    #don't forget to mark global variables as global
    global t
    t += dt
    if t < 3:
        return q
    print "Stopping push loop at t=3"
    return None

In [25]:
#run this cell to bind your code to the visualization playback window. You should 
#not need to modify this code.

kvis.add_text("HUD1",1,1)
kvis.add_text("HUD2",1,5)
kvis.add_text("HUD3",1,9)
kvis.add_text("HUD4",1,13)
kvis.add_text("HUD5",1,17)
kvis.add_text("HUD6",1,21)
kvis.add_text("HUD7",1,25)

#the amount of time between refreshes
dt = 0.05
                
def do_sense(button):
    output_area.clear_output()
    with output_area:
        update_objects_from_simulator()
    kvis.update()
    
def do_push(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            push_control_init(robot,robot.getConfig())
            while(True):
                robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
                qdes = push_control_loop(robot,robot.getConfig(),dt)
                if qdes is None:
                    print "Done."
                    break
                robot_control_api.setConfig(klampt_to_controller(qdes))
                wait_controller(dt)
        except Exception:
            import traceback
            traceback.print_exc()
        
def do_push_step(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            qdes = push_control_loop(robot,robot.getConfig(),dt)
            if qdes is None:
                print "Controller returned None."
            else:
                robot_control_api.setConfig(klampt_to_controller(qdes))
                wait_controller(dt)
        except Exception:
            import traceback
            traceback.print_exc()
            
def do_push_reset(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            push_control_init(robot,robot.getConfig())
        except Exception:
            import traceback
            traceback.print_exc()

sense_button = widgets.Button(description="Sense")
push_begin_button = widgets.Button(description="Push")
push_reset_button = widgets.Button(description="Reset push controller")
push_step_button = widgets.Button(description="Step push controller")
sense_button.on_click(do_sense)
push_begin_button.on_click(do_push)
push_reset_button.on_click(do_push_reset)
push_step_button.on_click(do_push_step)

controls.children = (push_begin_button,push_step_button,push_reset_button,sense_button)

## Assignment

Your job is to devise a controller that pushes the block from its initial position to the target position. The robot should not collide with the table, and the block should not fall off the table.  You are free to use any method that you wish to implement this controller, e.g., IK, motion planning, or heuristic methods.

In your report, precisely and in technical English language (not code, although pseudocode may be acceptable), describe your strategy for pushing the object.  This description should include, but is not limited to:

1. A high-level description of the controller, including a state machine or control flow diagram if applicable.

2. A detailed description of the implementation for any components that are not off-the-shelf.  Describe the mathematical models and assumptions made in these implementations.

3. For off-the-shelf components, describe their inputs and outputs.


## Unit testing

Perform unit testing of your controller, and describe the performance metrics and test procedure that you are using.  Interpret the observations you have made, including but not limited to:

* Whether performance is sensitive to certain parameters.
* Whether performance limitations or bottlenecks can be identified.
* Whether unit tests agree with theoretical behavior.

This should include, but is not limited to, an analysis of how well your strategy handles variation in the initial position of the block, initial position of the robot, and target location.